In [1]:
!pip install autokeras

     |████████████████████████████████| 166 kB 10.2 MB/s 
     |████████████████████████████████| 96 kB 4.7 MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-py3-none-any.whl size=9568 sha256=60a6324781991ec2b2a63b71fd769686627154628eb98beddc9ecc0944fa15d1
  Stored in directory: /root/.cache/pip/wheels/38/5c/e0/13003e68c17f403af40b92a24d20171b95fef13b0fdaba833c
Successfully built kt-legacy


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import autokeras as ak

In [3]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
data = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv")
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
data.rename( columns={'Unnamed: 0':'Date_time','468':'DMA_468'}, inplace=True )
data = data.replace(",", ".", regex=True)

In [5]:
# Dataset preparation
data_DMA225 = data[['Date_time','DMA_468','tempC','HeatIndexC','FeelsLikeC','WindChillC','windspeedKmph','sunHour','precipMM','humidity']]
data_DMA225_indexed = data_DMA225.set_index('Date_time')
# Making hourly data 
hourly_data = data_DMA225_indexed.resample('h').sum()
split = int(len(hourly_data) * 0.7)
trainset = hourly_data[:split]
validationset = hourly_data[split:]
testset = hourly_data[["tempC","HeatIndexC","FeelsLikeC","WindChillC","windspeedKmph","sunHour","precipMM","humidity"]]
X_train = trainset[["tempC","HeatIndexC","FeelsLikeC","WindChillC","windspeedKmph","sunHour","precipMM","humidity"]]
valid_X = validationset[['tempC','HeatIndexC','FeelsLikeC','WindChillC','windspeedKmph','sunHour','precipMM','humidity']]
y_train = trainset["DMA_468"]
valid_y = validationset["DMA_468"]

In [6]:
# Creating autokeras model:
model = ak.TimeseriesForecaster(
    predict_from=1,
    lookback=12,
    max_trials=1,
    loss='mse', optimizer='adam',
    objective="val_loss",
)

history = model.fit(X_train,y_train,batch_size=100,epochs = 100,validation_data=(valid_X, valid_y))

Trial 1 Complete [00h 00m 33s]
val_loss: 0.13597695529460907

Best val_loss So Far: 0.13597695529460907
Total elapsed time: 00h 00m 33s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
16/16 [==============================] - 8s 130ms/step - loss: 0.2889 - val_loss: 0.2279
Epoch 2/100
16/16 [==============================] - 1s 32ms/step - loss: 0.2240 - val_loss: 0.2136
Epoch 3/100
16/16 [==============================] - 1s 32ms/step - loss: 0.2075 - val_loss: 0.2102
Epoch 4/100
16/16 [==============================] - 1s 32ms/step - loss: 0.1966 - val_loss: 0.2060
Epoch 5/100
16/16 [==============================] - 1s 33ms/step - loss: 0.1885 - val_loss: 0.2015
Epoch 6/100
16/16 [==============================] - 1s 33ms/step - loss: 0.1818 - val_loss: 0.1973
Epoch 7/100
16/16 [==============================] - 0s 31ms/step - loss: 0.1748 - val_loss: 0.1916
Epoch 8/100
16/16 [==============================] - 0s 31ms/step - loss: 0.1664 - val_loss: 0.1857
Epoch 9/100
16/16 [======

INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


In [7]:
# Predicting on test set:
test_pred = model.predict(testset)
print(test_pred.shape)

22/22 [==============================] - 0s 18ms/step
(651, 1)


Autokeras is mainly used for mutlivaraite time series analysis. Here the validation loss is greater than train set loss. So it proves auto keras isnt advisble for time series prediction.